In [1]:
import pandas as pd
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField
from openff.units import unit

from openff.interchange import Interchange
from openff.interchange.drivers.gromacs import get_gromacs_energies
from openff.interchange.drivers.openmm import _get_openmm_energies, get_openmm_energies

In [2]:
SMILES = "c1n(CCO)c(C(F)(F)(F))cc1CNCCl"

In [3]:
molecule = Molecule.from_smiles(SMILES)
molecule.generate_conformers(n_conformers=20, rms_cutoff=0.1 * unit.angstrom)
topology = molecule.to_topology()

/Users/mwt/miniconda3/envs/openff-interchange-env/lib/python3.9/site-packages/openff/toolkit/utils/openeye_wrapper.py:1032: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  positions[off_atom_index, :] = off_atom_coords


In [4]:
parsley = ForceField("openff-1.1.0.offxml")

In [5]:
openff_sys = Interchange.from_smirnoff(force_field=parsley, topology=topology)
openmm_sys = parsley.create_openmm_system(topology)

In [6]:
openff_sys.positions = molecule.conformers[0]
openff_sys.box = [4, 4, 4]

In [7]:
df = pd.DataFrame()

kj_mol = unit.kilojoule / unit.mol

for idx, conformer in enumerate(molecule.conformers):
    openff_sys.positions = conformer

    toolkit = sum(
        _get_openmm_energies(
            omm_sys=openmm_sys,
            box_vectors=openff_sys.box,
            positions=openff_sys.positions,
        ).energies.values()
    ).m_as(kj_mol)

    omm = sum(get_openmm_energies(openff_sys).energies.values()).m_as(kj_mol)

    gmx = sum(get_gromacs_energies(openff_sys).energies.values()).m_as(kj_mol)

    df = pd.concat(
        [
            df,
            pd.DataFrame.from_dict(
                {
                    "Conformer No.": [idx],
                    "Toolkit (kJ/mol)": [round(toolkit, 3)],
                    "Interchange -> OpenMM (kJ/mol)": [round(omm, 3)],
                    "Interchange -> GROMACS": [round(gmx, 3)],
                }
            ),
            # ignore_index=True,
        ]
    )

In [8]:
df.style.hide(axis="index")

Conformer No.,Toolkit (kJ/mol),Interchange -> OpenMM (kJ/mol),Interchange -> GROMACS
0,379.534000,379.457000,379.375000
1,378.628000,378.592000,378.522000
2,393.001000,392.866000,392.794000
3,388.423000,388.405000,388.341000
4,388.387000,388.351000,388.293000
5,387.330000,387.251000,387.174000
6,373.601000,373.541000,373.480000
7,372.383000,372.334000,372.284000
8,399.647000,399.568000,399.509000
9,399.208000,399.136000,399.059000
